In [22]:
!nvidia-smi

Sun Apr 14 19:53:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [23]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [24]:

!git clone https://github.com/NVIDIA/cuda-samples.git

fatal: destination path 'cuda-samples' already exists and is not an empty directory.


In [25]:
!cd cuda-samples/Samples/1_Utilities/deviceQuery && make


make: Nothing to be done for 'all'.


In [26]:
!cd cuda-samples/Samples/1_Utilities/deviceQuery && ls
!cuda-samples/Samples/1_Utilities/deviceQuery/./deviceQuery

deviceQuery	 deviceQuery_vs2017.sln      deviceQuery_vs2019.vcxproj  Makefile
deviceQuery.cpp  deviceQuery_vs2017.vcxproj  deviceQuery_vs2022.sln	 NsightEclipse.xml
deviceQuery.o	 deviceQuery_vs2019.sln      deviceQuery_vs2022.vcxproj  README.md
cuda-samples/Samples/1_Utilities/deviceQuery/./deviceQuery Starting...

 CUDA Device Query (Runtime API) version (CUDART static linking)

Detected 1 CUDA Capable device(s)

Device 0: "Tesla T4"
  CUDA Driver Version / Runtime Version          12.2 / 12.2
  CUDA Capability Major/Minor version number:    7.5
  Total amount of global memory:                 15102 MBytes (15835660288 bytes)
  (040) Multiprocessors, (064) CUDA Cores/MP:    2560 CUDA Cores
  GPU Max Clock rate:                            1590 MHz (1.59 GHz)
  Memory Clock rate:                             5001 Mhz
  Memory Bus Width:                              256-bit
  L2 Cache Size:                                 4194304 bytes
  Maximum Texture Dimension Size (x,y,z)         1D=

In [27]:
!pip install nvcc4jupyter

In [28]:
%load_ext nvcc4jupyter

The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


In [30]:
%%writefile NOCUDA.cpp

#include <iostream>
#include <vector>
#include <ctime>
#include <cstdlib>
#include <cmath>

#define GENOME_LENGTH 16
#define NUMBER_CHROMOSOMES 1024
#define GENE_MIN -1
#define GENE_MAX 1
#define MUTATION_FACTOR 0.2
#define CROSSOVER_RATE 0.6
#define NUM_EPOCHS 1000

using namespace std;

typedef long double HighlyPrecise;

typedef struct {
    HighlyPrecise genes[GENOME_LENGTH];
    HighlyPrecise fitnessValue;
} Chromosome;

HighlyPrecise geneValueDistribution() {
    return ((HighlyPrecise)rand() / RAND_MAX) * (GENE_MAX - GENE_MIN) + GENE_MIN;
}

void initializePopulation(vector<Chromosome>& population) {
    for (int i = 0; i < NUMBER_CHROMOSOMES; i++) {
        Chromosome chromosome;
        for (int j = 0; j < GENOME_LENGTH; j++) {
            chromosome.genes[j] = geneValueDistribution();
        }
        population.push_back(chromosome);
    }
}

HighlyPrecise getFitnessValue(Chromosome chromosome) {
    HighlyPrecise value = 0.0;
    for (int i = 0; i < GENOME_LENGTH; i++) {
        value += chromosome.genes[i] * chromosome.genes[i];
    }
    return value;
}

Chromosome crossover(Chromosome male, Chromosome female) {
    int mid = GENOME_LENGTH / 2;
    Chromosome offspring;
    for (int i = 0; i < GENOME_LENGTH; i++) {
        offspring.genes[i] = (i <= mid) ? male.genes[i] : female.genes[i];
    }
    return offspring;
}

void mutate(Chromosome& chromosome) {
    for (int i = 0; i < GENOME_LENGTH; i++) {
        HighlyPrecise multiplier = ((HighlyPrecise)rand() / RAND_MAX) * 0.2 - 0.1;
        chromosome.genes[i] *= multiplier;
    }
}

void startIteration(vector<Chromosome>& population) {
    int num_parents = (int)(NUMBER_CHROMOSOMES * (1 - CROSSOVER_RATE));
    int num_offsprings = NUMBER_CHROMOSOMES - num_parents;

    for (int i = num_parents; i < NUMBER_CHROMOSOMES; i++) {
        int fatherIndex = rand() % num_parents;
        int motherIndex = rand() % num_parents;
        if (motherIndex == fatherIndex) {
            continue;
        }

        Chromosome male = population[fatherIndex];
        Chromosome female = population[motherIndex];
        Chromosome offspring = crossover(male, female);

        if (MUTATION_FACTOR >= ((HighlyPrecise)rand() / RAND_MAX)) {
            mutate(offspring);
        }
        population[i] = offspring;
    }
}

void geneticAlgorithm(vector<Chromosome>& population, Chromosome& best) {
    clock_t start = clock();

    for (int z = 0; z < NUM_EPOCHS; z++) {
        startIteration(population);
    }

    clock_t end = clock();
    double time_spent = (double)(end - start) / CLOCKS_PER_SEC;
    cout << "Execution time (s): " << time_spent << endl;

    best = population[0];
}

void printChromosome(Chromosome chromosome) {
    cout << "Chromosome: ";
    for (int i = 0; i < GENOME_LENGTH; i++) {
        cout << chromosome.genes[i] << " ";
    }
    cout << endl << "Fitness: " << chromosome.fitnessValue << endl;
}

int main() {
    srand(time(NULL));

    vector<Chromosome> population;
    Chromosome best;

    initializePopulation(population);
    geneticAlgorithm(population, best);

    cout << "==== CPU Results ====" << endl;
    printChromosome(best);

    return 0;
}


Writing NOCUDA.cpp


In [35]:
! g++ NOCUDA.cpp -o NOCUDA

In [36]:
 ! ./NOCUDA

Execution time (s): 0.232576
==== CPU Results ====
Chromosome: -0.781291 -0.904536 -0.581516 -0.0676536 0.730351 0.247694 -0.463701 0.0100267 0.14065 -0.837937 -0.093677 -0.119095 -0.704239 -0.355675 0.927895 0.267829 
Fitness: nan


In [37]:
%%cuda
#include <iostream>
#include <vector>
#include <curand_kernel.h>
#include <cmath> // For pow function

#define GENOME_LENGTH 16
#define NUMBER_CHROMOSOMES 1024
#define GENE_MIN -1
#define GENE_MAX 1
#define MUTATION_FACTOR 0.2
#define CROSSOVER_RATE 0.6
#define NUM_EPOCHS 1000

using namespace std;

typedef long double HighlyPrecise;

typedef struct {
    HighlyPrecise genes[GENOME_LENGTH];
    HighlyPrecise fitnessValue;
} Chromosome;

__device__ HighlyPrecise getFitnessValue(Chromosome chromosome) {
    HighlyPrecise value = 0.0;
    for (int i = 0; i < GENOME_LENGTH; i++) {
        value += chromosome.genes[i] * chromosome.genes[i];
    }
    return value;
}

__device__ HighlyPrecise geneValueDistribution(curandState* state) {
    return ((HighlyPrecise)curand_uniform(state) / RAND_MAX) * (GENE_MAX - GENE_MIN) + GENE_MIN;
}

__global__ void initializePopulation(Chromosome* population, curandState* states) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    curandState localState = states[idx];
    for (int j = 0; j < GENOME_LENGTH; j++) {
        population[idx].genes[j] = geneValueDistribution(&localState);
    }
    population[idx].fitnessValue = getFitnessValue(population[idx]);
}

__device__ Chromosome crossover(Chromosome male, Chromosome female, curandState* state) {
    int mid = GENOME_LENGTH / 2;
    Chromosome offspring;
    for (int i = 0; i < GENOME_LENGTH; i++) {
        offspring.genes[i] = (i <= mid) ? male.genes[i] : female.genes[i];
    }
    return offspring;
}

__device__ void mutate(Chromosome* chromosome, curandState* state) {
    for (int i = 0; i < GENOME_LENGTH; i++) {
        HighlyPrecise multiplier = ((HighlyPrecise)curand_uniform(state) / RAND_MAX) * 0.2 - 0.1;
        chromosome->genes[i] *= multiplier;
    }
}

__global__ void startIteration(Chromosome* population, curandState* states) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    int num_parents = (int)(NUMBER_CHROMOSOMES * (1 - CROSSOVER_RATE));
    curandState localState = states[idx];

    if (idx >= num_parents) {
        int fatherIndex = curand_uniform(&localState) * num_parents;
        int motherIndex = curand_uniform(&localState) * num_parents;
        if (motherIndex == fatherIndex) {
            return;
        }

        Chromosome male = population[fatherIndex];
        Chromosome female = population[motherIndex];
        Chromosome offspring = crossover(male, female, &localState);

        if (MUTATION_FACTOR >= curand_uniform(&localState)) {
            mutate(&offspring, &localState);
        }
        population[idx] = offspring;
    }
}

void geneticAlgorithm(Chromosome* d_population, Chromosome* d_best, curandState* d_states) {
    clock_t start = clock();

    for (int z = 0; z < NUM_EPOCHS; z++) {
        startIteration<<<1, NUMBER_CHROMOSOMES>>>(d_population, d_states);
        cudaDeviceSynchronize();
    }

    clock_t end = clock();
    double time_spent = (double)(end - start) / CLOCKS_PER_SEC;
    printf("Execution time (s): %.10lf\n", time_spent);

    cudaMemcpy(d_best, d_population, sizeof(Chromosome), cudaMemcpyDeviceToDevice);
}

void printChromosome(Chromosome chromosome) {
    cout << "Chromosome: ";
    for (int i = 0; i < GENOME_LENGTH; i++) {
        cout << chromosome.genes[i] << " ";
    }
    cout << endl << "Fitness: " << chromosome.fitnessValue << endl;
}

int main() {
    srand(time(NULL));

    Chromosome* d_population, * d_best;
    Chromosome* population = new Chromosome[NUMBER_CHROMOSOMES];
    Chromosome best;

    curandState* d_states;

    // Loop over different values of N (threads per block)
    for (int N = 2; N <= 20; N++) {
        cudaMalloc(&d_population, NUMBER_CHROMOSOMES * sizeof(Chromosome));
        cudaMalloc(&d_best, sizeof(Chromosome));
        cudaMalloc(&d_states, NUMBER_CHROMOSOMES * N * sizeof(curandState));

        initializePopulation<<<NUMBER_CHROMOSOMES, N>>>(d_population, d_states);
        cudaDeviceSynchronize();

        geneticAlgorithm(d_population, d_best, d_states);
        cudaMemcpy(&best, d_best, sizeof(Chromosome), cudaMemcpyDeviceToHost);

        cout << "==== GPU Results (N = " << N << ") ====" << endl;
        printChromosome(best);

        cudaFree(d_population);
        cudaFree(d_best);
        cudaFree(d_states);
    }

    delete[] population;

    return 0;
}


Execution time (s): 0.0004320000
==== GPU Results (N = 2) ====
Chromosome: 0 0 0 0 0 7.2904e-4951 0 0 0 0 0 0 0 0 0 0 
Fitness: 0
Execution time (s): 0.0004440000
==== GPU Results (N = 3) ====
Chromosome: 0 0 0 0 0 7.2904e-4951 0 0 0 0 0 0 0 0 0 0 
Fitness: 0
Execution time (s): 0.0004740000
==== GPU Results (N = 4) ====
Chromosome: 0 0 0 0 0 7.2904e-4951 0 0 0 0 0 0 0 0 0 0 
Fitness: 0
Execution time (s): 0.0004310000
==== GPU Results (N = 5) ====
Chromosome: 0 0 0 0 0 7.2904e-4951 0 0 0 0 0 0 0 0 0 0 
Fitness: 0
Execution time (s): 0.0005030000
==== GPU Results (N = 6) ====
Chromosome: 0 0 0 0 0 7.2904e-4951 0 0 0 0 0 0 0 0 0 0 
Fitness: 0
Execution time (s): 0.0004370000
==== GPU Results (N = 7) ====
Chromosome: 0 0 0 0 0 7.2904e-4951 0 0 0 0 0 0 0 0 0 0 
Fitness: 0
Execution time (s): 0.0004370000
==== GPU Results (N = 8) ====
Chromosome: 0 0 0 0 0 7.2904e-4951 0 0 0 0 0 0 0 0 0 0 
Fitness: 0
Execution time (s): 0.0004260000
==== GPU Results (N = 9) ====
Chromosome: 0 0 0 0 0 7.290